In [1]:
# Import packages
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

# snv

In [11]:
# Import data
data = pd.read_csv(r".\snv.csv")
label = pd.read_csv(r'.\wr_label.csv')
enc = OneHotEncoder().fit_transform(label)
enc.toarray()
data.columns = np.double(data.columns)
label = enc.toarray()
x = np.array(data)
y = np.array(label)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.7,random_state = 300)
y_train = np.array(y_train)

In [12]:
# training model
clf =ak.StructuredDataClassifier(overwrite = True,loss="categorical_crossentropy",metrics="accuracy",max_trials=50)   #Set the parameters
clf.fit(x_train,y_train,epochs=300)
model=clf.export_model()
model.save("./model/model_autokeras5.h5")    # save model 
model.summary()    # print model 

Trial 32 Complete [00h 00m 31s]
val_accuracy: 0.6571428775787354

Best val_accuracy So Far: 0.6857143044471741
Total elapsed time: 00h 32m 41s
INFO:tensorflow:Oracle triggered exit

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/300
11/11 [==============================] - 0s 2ms/step - loss: 1.3537 - accuracy: 0.4520
Epoch 2/300
11/11 [==============================] - 0s 2ms/step - loss: 0.9393 - accuracy: 0.6378
Epoch 3/300
11/11 [==============================] - 0s 2ms/step - loss: 0.8402 - accuracy: 0.6935
Epoch 4/300
11/11 [===========

In [13]:
from tensorflow.keras.models import load_model
loaded_model = load_model("./model/model_autokeras5.h5", custom_objects=ak.CUSTOM_OBJECTS)   # import model
model.summary()
# predict
y_train_predicted = loaded_model.predict(x_train)
y_test_predicted = loaded_model.predict(x_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
# Call evaluation indicators
from sklearn.metrics import confusion_matrix
print(metrics.classification_report(y_train, clf.predict(x_train),digits=4))
print(metrics.accuracy_score(y_train,clf.predict(x_train)))
print(confusion_matrix(y_train.argmax(axis=1),y_train_predicted.argmax(axis=1)))
print(metrics.classification_report(y_test, clf.predict(x_test),digits=4))
print(metrics.accuracy_score(y_test,clf.predict(x_test)))
print(confusion_matrix(y_test.argmax(axis=1),y_test_predicted.argmax(axis=1)))

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 228)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 228)               0         
_________________________________________________________________
normalization (Normalization (None, 228)               457       
_________________________________________________________________
dense (Dense)                (None, 32)                7328      
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              